In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Reshape, Conv2D, MaxPooling2D, Flatten
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.metrics import confusion_matrix
import random
import timeit
from sklearn.utils import class_weight
start = timeit.default_timer()


#Constant
EMOTION_ANNOTATORS = {'anger': 0, 'happiness' : 1, 'sadness' : 2, 'neutral' : 3, 'frustration' : 4, 'excited': 5,
           'fear' : 6,'surprise' : 7,'disgust' : 8, 'other' : 9}

EMOTION = {'ang': 0, 'hap' : 1, 'sad' : 2, 'neu' : 3, 'fru' : 4, 'exc': 5,
           'fea' : 6,'sur' : 7,'dis' : 8, 'oth' : 9, 'xxx':10}

METHOD = {'audio_feature':0, 'LSTM':1, 'DRCNN' : 2}

#Method for classification
method = METHOD['DRCNN']

#If data is processed and saved into files, just reload, dont need to re-process
isRawDataProcessed = True

#Development mode. Only run with small data.
dev = False








In [ ]:
##Load the array of input files and output lables. Then, shuffle data, split into train and test



##Function to transform these above array into data used for training and testing


In [ ]:
num_class = 8
batch_size = 1000
width_image = 256
channels = 3 #RGB


if method == METHOD['DRCNN']: 
    
       
    model = Sequential()
    
    #C1,P1
    model.add(Conv2D(4, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=(batch_size, width_image, width_image, channels)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    #C2,P2
    model.add(Conv2D(8, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    #C3
    model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu'))
    
    #C4
    model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu'))
    
    #C5, P5
    model.add(Conv2D(64, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    
    #F6, F7, F8
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_class activation='softmax'))
    
    
    #C2,P2
    model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    
    
    
  
    #Your statements here

    stop = timeit.default_timer()

    print ("Time for training and testing: ", stop - start, "(s)") 